In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
import datetime
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [2]:
train_data=pd.read_csv("../../data_p2/ml_case_training_data.csv", encoding='utf-8')
train_hist=pd.read_csv("../../data_p2/ml_case_training_hist_data.csv", encoding='cp949')
train_out=pd.read_csv("../../data_p2/ml_case_training_output.csv", encoding='utf-8')

In [3]:
#결측치가 많은 7개 칼럼 삭제
nan_count=((train_data.isna().sum()/train_data.shape[0])*100).sort_values(ascending=False)
nan_count=nan_count[nan_count>0]
train_data.drop(nan_count.index[:7],axis=1,inplace=True)

In [4]:
#date_modif_prod 결측치 새 카테고리 만들어서 채우기
#'date_modif_prod','date_renewal','date_end' 는 bfill로 채움
train_data['channel_sales']=train_data['channel_sales'].fillna('new category')
train_data[['date_modif_prod','date_renewal','date_end']]=train_data[['date_modif_prod','date_renewal',
                                                            'date_end']].fillna(method='bfill')

/tmp/ipykernel_81443/2131512965.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  'date_end']].fillna(method='bfill')


In [5]:
#'forecast_discount_energy','forecast_price_pow_p1','margin_gross_pow_ele','margin_net_pow_ele','net_margin','pow_max'
# 정수변환 및 최빈값 채우기
for col in [x for x in nan_count.index[8:] if x not in ['date_modif_prod','date_renewal','date_end','origin_up','forecast_price_energy_p1','forecast_price_energy_p2']]:
    train_data[col] = np.round(pd.to_numeric(train_data[col], errors='coerce')).astype('Int64')

In [6]:
#origin_up 결측치 최빈값 채우기
origin_imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
train_data[['origin_up']]=origin_imp.fit_transform(train_data[['origin_up']])

In [7]:

imp_freq = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
train_data[
    [x for x in nan_count.index[8:] if x not in ['date_modif_prod', 'date_renewal', 'origin_up', 'date_end']]
] = imp_freq.fit_transform(
    train_data[
        [x for x in nan_count.index[8:] if x not in ['date_modif_prod', 'date_renewal', 'origin_up', 'date_end']]
    ]
)

In [8]:
train=pd.merge(train_data, train_out, on=['id'])

In [10]:
train.to_csv('전처리v1.csv', index=False)

In [14]:
train.head()
train.shape

(16096, 26)

In [15]:
#날짜 변환
import pandas as pd

datetype_col = ["date_activ", "date_end", "date_modif_prod", "date_renewal"]
base_date = pd.to_datetime('2000-01-01')

for col in datetype_col:
    train[col] = pd.to_datetime(train[col], errors='coerce')  # 문자열 → datetime, 결측은 NaT로 유지
    train[col] = (train[col] - base_date).dt.days             # 경과일수, 결측은 NaN
    train[col] = train[col].astype('Int64')                   # 결측치는 <NA>로, 나머지는 int형

In [18]:
train[["date_activ", "date_end", "date_modif_prod", "date_renewal"]].min()

date_activ          206
date_end           2429
date_modif_prod     206
date_renewal       4925
dtype: Int64

In [19]:
train[["date_activ", "date_end", "date_modif_prod", "date_renewal"]] = train[["date_activ", "date_end", "date_modif_prod", "date_renewal"]] - 206

In [21]:
train.to_csv('전처리v1.csv', index=False)